<a href="https://colab.research.google.com/github/Xalp/NLP1d/blob/master/NLP_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/Xalp/NLP1d

Cloning into 'NLP1d'...
remote: Enumerating objects: 23, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 23 (delta 1), reused 17 (delta 0), pack-reused 0
Unpacking objects: 100% (23/23), done.


In [ ]:
import os
os.chdir("/content/NLP1d")

## Part 1

In [ ]:
# Emission probability
from collections import defaultdict
import numpy as np
def emission(train):
    '''
    x: str, word
    y: str, tag
    train: training set path
    '''
    count_y_x = defaultdict(float)
    count_y  = defaultdict(float)
    emission_probs = defaultdict(float)
    dict_f = defaultdict()
    with open(train) as f:
        for line in f.readlines():
            if line == '\n': continue
            x, y = line.split()
            count_y_x[(y,x)] += 1
    
    for (y,x), cnt in count_y_x.items():
        count_y[y] += cnt
    for (y,x) in count_y_x.keys():
        e_x_y = count_y_x[(y,x)]/count_y[y]
        emission_probs[(x,y)] = e_x_y
        str_x_y = 'emission:'+str(y)+'+'+str(x)
        dict_f[str_x_y] = np.log(e_x_y)

    return emission_probs, dict_f

train = '/content/NLP1d/dataset/train'
emission_probs, dict_f = emission(train)
print(emission_probs, dict_f)

defaultdict(<class 'float'>, {('All', 'O'): 0.00012124641312694499, ('in', 'O'): 0.01066968435517116, ('all', 'O'): 0.00307157579921594, (',', 'O'): 0.038879683142707025, ('the', 'O'): 0.04542698945156206, ('food', 'B-positive'): 0.1096234309623431, ('was', 'O'): 0.01794446914278786, ('great', 'O'): 0.00614315159843188, ('(', 'O'): 0.00315240674130057, ('except', 'O'): 0.00016166188416926, ('for', 'O'): 0.01147799377601746, ('dessserts', 'B-negative'): 0.0026455026455026454, (')', 'O'): 0.00339489956755446, ('.', 'O'): 0.060946530331811015, ('I', 'O'): 0.019601503455522775, ('have', 'O'): 0.006345228953643455, ('NEVER', 'O'): 0.000202077355211575, ('been', 'O'): 0.002142019965242695, ('disappointed', 'O'): 0.000525401123550095, ('Red', 'B-positive'): 0.0016736401673640166, ('Eye', 'I-positive'): 0.001736111111111111, ('Great', 'O'): 0.001414541486481025, ('with', 'O'): 0.0059006587721779896, ('an', 'O'): 0.002142019965242695, ('awesome', 'O'): 0.00028290829729620497, ('atmosphere', 'B-

In [ ]:
import numpy as np
# Transition probability
def transition(train):
  with open(train,"r") as f:
    data = f.read().rstrip().splitlines()

  count_bigram = {}
  count_unigram = {}

  count_unigram["START"] = 0
  count_unigram["STOP"] = 0

  for i in range(len(data)):
    inst = data[i]
    # if this is an empty line:
    if len(inst) == 0:
      # building with previous inst
      tag = "STOP"
      if i != 0:
        prev_inst = data[i-1]
        prev_tag = prev_inst.split()[-1]
        if (tag, prev_tag) not in count_bigram.keys():
          count_bigram[(tag, prev_tag)] = 1
        else:
          count_bigram[(tag, prev_tag)] += 1
      # building with the next inst
      tag = "START"
      if i != len(data) - 1:
        next_inst = data[i+1]
        next_tag = next_inst.split()[-1]
        if (tag, next_tag) not in count_bigram.keys():
          count_bigram[(tag, next_tag)] = 1
        else:
          count_bigram[(tag, next_tag)] += 1
      # increase the count of start and stop
      count_unigram["START"] += 1
      count_unigram["STOP"] += 1
      continue
    # do in all cases:
    tag = inst.split()[-1]
    if tag not in count_unigram.keys():
      count_unigram[tag] = 1
    else:
      count_unigram[tag] += 1
    # if not the last inst:
    if i != len(data) - 1:
      next_inst = data[i+1]
      if len(next_inst) == 0:
        continue
      next_tag = next_inst.split()[-1]
      if (tag, next_tag) not in count_bigram.keys():
        count_bigram[(tag, next_tag)] = 1
      else:
        count_bigram[(tag, next_tag)] += 1

  # calculate transition matrix
  f = {}
  q = {}
  for (y_1, y_2), count_bi in count_bigram.items():
    _str_ = "transition:" + y_1 + "+" + y_2
    q[(y_1, y_2)] = count_bi / count_unigram[y_1]
    if count_bi == 0:
      f[_str_] = - np.inf
    else:
      f[_str_] = np.log(count_bi / count_unigram[y_1])
  return q, f

q,f = transition("/content/NLP1d/dataset/train")
print(q)
print(f)

{('O', 'O'): 0.863355292405933, ('O', 'B-positive'): 0.045022834741138905, ('B-positive', 'O'): 0.699581589958159, ('O', 'B-negative'): 0.01446873863314877, ('B-negative', 'O'): 0.791005291005291, ('STOP', 'O'): 0.9845085470085471, ('START', 'O'): 0.9401709401709402, ('B-positive', 'I-positive'): 0.28535564853556483, ('I-positive', 'O'): 0.5815972222222222, ('O', 'B-neutral'): 0.002627005617750475, ('B-neutral', 'O'): 0.7922077922077922, ('START', 'B-negative'): 0.010683760683760684, ('START', 'B-positive'): 0.042735042735042736, ('I-positive', 'I-positive'): 0.4079861111111111, ('B-neutral', 'I-neutral'): 0.16883116883116883, ('I-neutral', 'I-neutral'): 0.48, ('I-neutral', 'O'): 0.52, ('B-negative', 'I-negative'): 0.20105820105820105, ('I-negative', 'O'): 0.5984251968503937, ('START', 'B-neutral'): 0.00641025641025641, ('I-negative', 'I-negative'): 0.4015748031496063, ('STOP', 'B-positive'): 0.009081196581196582, ('STOP', 'I-positive'): 0.003205128205128205, ('B-positive', 'B-positive

## part 2